In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.metrics import r2_score
from sklearn import preprocessing

In [23]:
import warnings
warnings.filterwarnings('ignore')

In [24]:
from keras.models import Sequential
from keras.layers import Dense

In [25]:
voice_data = pd.read_csv('../datasets/voice.csv')

voice_data.head()

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,...,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,label
0,0.059781,0.064241,0.032027,0.015071,0.090193,0.075122,12.863462,274.402906,0.893369,0.491918,...,0.059781,0.084279,0.015702,0.275862,0.007812,0.007812,0.007812,0.000000,0.000000,male
1,0.066009,0.067310,0.040229,0.019414,0.092666,0.073252,22.423285,634.613855,0.892193,0.513724,...,0.066009,0.107937,0.015826,0.250000,0.009014,0.007812,0.054688,0.046875,0.052632,male
2,0.077316,0.083829,0.036718,0.008701,0.131908,0.123207,30.757155,1024.927705,0.846389,0.478905,...,0.077316,0.098706,0.015656,0.271186,0.007990,0.007812,0.015625,0.007812,0.046512,male
3,0.151228,0.072111,0.158011,0.096582,0.207955,0.111374,1.232831,4.177296,0.963322,0.727232,...,0.151228,0.088965,0.017798,0.250000,0.201497,0.007812,0.562500,0.554688,0.247119,male
4,0.135120,0.079146,0.124656,0.078720,0.206045,0.127325,1.101174,4.333713,0.971955,0.783568,...,0.135120,0.106398,0.016931,0.266667,0.712812,0.007812,5.484375,5.476562,0.208274,male


In [26]:
voice_data = voice_data.sample(frac = 1).reset_index(drop = True)

len(voice_data)

3168

In [27]:
from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()

voice_data["label"] = lb_make.fit_transform(voice_data["label"])

In [28]:
x_data = voice_data.drop(columns=['label','sd','Q25','Q75','kurt','sfm','centroid', 'maxdom','mindom','dfrange'])

In [29]:
ss = preprocessing.StandardScaler()

x_data = ss.fit_transform(x_data)

x_data[:10]

array([[ 0.2131808 , -0.06736103, -0.88239266, -0.12882644, -0.71502588,
        -0.04388505,  0.41017264,  0.71418878,  0.6189829 ,  1.67116984,
        -0.27264338],
       [-0.86680684, -1.09144278,  0.43190829, -0.04887506,  0.56676271,
        -0.79613791, -1.16456765, -0.13296788,  0.14350786,  0.7158595 ,
         2.51311745],
       [-2.09607318, -2.17745509,  0.59877428, -0.70720083,  1.76327771,
        -1.67587485,  1.56542697, -0.55904782, -0.29403034, -0.87013729,
         1.16253655],
       [-0.03104542, -0.11190294, -1.27751275, -0.069143  , -0.94216612,
         0.28366434,  0.76150967,  0.77073205, -0.54594628, -0.15349962,
        -0.20292422],
       [ 1.48147125,  0.98464341, -0.98962355, -0.1425559 , -1.09887226,
         0.61965752,  1.03437599,  0.55067099,  0.67262382, -0.07605338,
        -0.83260606],
       [-0.00382558,  0.25397539, -0.01871595, -0.29734117,  0.46294081,
         0.50507567, -0.67269269,  0.56536262,  0.67262382,  1.03042988,
        -0.178

In [30]:
x_data = pd.DataFrame(x_data)

y_data = voice_data['label']

In [31]:
x_data = np.array(x_data)
y_data = np.array(y_data)

In [32]:
#80:20 Train:Test Split
train_data = x_data[:-633]
test_data = x_data[-633:]

train_labels = y_data[:-633]
test_labels = y_data[-633:]

In [33]:
model = Sequential()

In [34]:
model.add(Dense(64,
                name = 'Input_Layer',
                input_shape = (11,),
                activation = 'relu'
               )
         )

In [35]:
model.add(Dense(2,
                name = 'Output_Layer',
                activation = 'softmax'
               )
         )

In [36]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ Input_Layer (Dense)                  │ (None, 64)                  │             768 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Output_Layer (Dense)                 │ (None, 2)                   │             130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 898 (3.51 KB)

 Trainable params: 898 (3.51 KB)

 Non-trainable params: 0 (0.00 B)

In [41]:
model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

In [42]:
model.fit(train_data,
          train_labels,
          epochs = 30,
          batch_size = 5,
          verbose = 1)

Epoch 1/30
507/507 ━━━━━━━━━━━━━━━━━━━━ 1s 795us/step - accuracy: 0.8775 - loss: 0.3357  
Epoch 2/30
507/507 ━━━━━━━━━━━━━━━━━━━━ 0s 790us/step - accuracy: 0.9745 - loss: 0.0906
Epoch 3/30
507/507 ━━━━━━━━━━━━━━━━━━━━ 0s 758us/step - accuracy: 0.9767 - loss: 0.0728
Epoch 4/30
507/507 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9802 - loss: 0.0653 
Epoch 5/30
507/507 ━━━━━━━━━━━━━━━━━━━━ 0s 767us/step - accuracy: 0.9804 - loss: 0.0591
Epoch 6/30
507/507 ━━━━━━━━━━━━━━━━━━━━ 0s 745us/step - accuracy: 0.9758 - loss: 0.0660
Epoch 7/30
507/507 ━━━━━━━━━━━━━━━━━━━━ 0s 756us/step - accuracy: 0.9816 - loss: 0.0606
Epoch 8/30
507/507 ━━━━━━━━━━━━━━━━━━━━ 0s 753us/step - accuracy: 0.9826 - loss: 0.0602
Epoch 9/30
507/507 ━━━━━━━━━━━━━━━━━━━━ 0s 779us/step - accuracy: 0.9759 - loss: 0.0687
Epoch 10/30
507/507 ━━━━━━━━━━━━━━━━━━━━ 0s 765us/step - accuracy: 0.9819 - loss: 0.0503
Epoch 11/30
507/507 ━━━━━━━━━━━━━━━━━━━━ 0s 753us/step - accuracy: 0.9828 - loss: 0.0482
Epoch 12/30
507/507 ━━━━━━━━━

In [44]:
loss,accuracy = model.evaluate(test_data,
                              test_labels,
                              verbose = 1)

20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9864 - loss: 0.0533 


In [46]:
print('loss = {:2f}'.format(loss))
print('Accuracy = {:2f}'.format(accuracy))

loss = 0.060315
Accuracy = 0.981043
